In [ ]:
function firing_rates(t_grid, θ::PoissonProcessParams, KernList::AbstractArray; n_range=[], d_range=[], m_range=[])
    C, α, u, N, M, D, J = name_params(θ)
    b = θ.b
    
    if isempty(n_range) n_range = 1:N end
    if isempty(m_range) m_range = 1:M end
    if isempty(d_range) d_range = 1:D end
    
    latent_funcs = Array(Any,(maximum(n_range), maximum(d_range)))
    for n = n_range
        for d = d_range
            tmp = function(t::Float64)
                out = 0
                for j = 1:length(u[d])
                     out += (α[n,d][j] * kernel(KernList[d], t, u[d][j]));
                end
                return out
            end
            latent_funcs[n, d] = tmp
        end
    end
    
    firing_rate = Array(Any, (maximum(n_range), maximum(m_range)))
    for n = n_range
        for m = m_range
            firing_rate[n,m] = zeros(length(t_grid))
            for t = 1:length(t_grid)
                for d = d_range
                    firing_rate[n,m][t] += C[m,d]*latent_funcs[n,d](t_grid[t]);
                end
            end
            firing_rate[n,m] = exp(firing_rate[n,m]+b[m]) # exponential link function
        end
    end
                
    return firing_rate
end